In [3]:
!pip install langchain openai duckduckgo-search youtube_search wikipedia langchainhub


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!pip install gradio==3.43.1

   ---------------------------------------- 0.0/20.1 MB ? eta -:--:--
    --------------------------------------- 0.4/20.1 MB 8.7 MB/s eta 0:00:03
   -- ------------------------------------- 1.0/20.1 MB 13.3 MB/s eta 0:00:02
   -- ------------------------------------- 1.2/20.1 MB 8.6 MB/s eta 0:00:03
   ---- ----------------------------------- 2.1/20.1 MB 12.2 MB/s eta 0:00:02
   ---- ----------------------------------- 2.1/20.1 MB 12.2 MB/s eta 0:00:02
   ---- ----------------------------------- 2.1/20.1 MB 12.2 MB/s eta 0:00:02
   ----- ---------------------------------- 2.8/20.1 MB 8.4 MB/s eta 0:00:03
   ------ --------------------------------- 3.1/20.1 MB 9.1 MB/s eta 0:00:02
   ------ --------------------------------- 3.1/20.1 MB 9.1 MB/s eta 0:00:02
   -------- ------------------------------- 4.2/20.1 MB 8.9 MB/s eta 0:00:02
   -------- ------------------------------- 4.2/20.1 MB 8.9 MB/s eta 0:00:02
   ---------- ----------------------------- 5.2/20.1 MB 9.3 MB/s eta 0:00:02
  


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Ensure you take care of imports, setting your OpenAI key, etc.
#%%capture

import os
import getpass
from prompter_245 import prompt

#os.environ["OPENAI_API_KEY"] = getpass.getpass("")

from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI 
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description
import gradio as gr


In [2]:
# Create toolkit
search = DuckDuckGoSearchAPIWrapper()

search_tool = Tool(name="Current Search",
                   func=search.run,
                   description="Useful when you need to answer questions about current events or the current state of the world."
                   )




tools = [search_tool]

# Set up memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize LLM
llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature=0, openai_api_key="" )
print('llm initialized')

# Initialize agent
agent_chain = initialize_agent(tools,
                               llm,
                               agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                               memory=memory,
                               verbose=True)

# Run queries
# Here is an example query:
agent_chain.run(input="What it do, nephew!")
agent_prompt = hub.pull("hwchase17/react-chat")

# Setting up an agent using LCEL
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

# Set up the agent using LCEL
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

memory = ConversationBufferMemory(memory_key="chat_history")

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

# # Here are some example queries:
# agent_executor.invoke({"input": "What's the forecast for snow looking like in Winnipeg today?"})["output"]
# agent_executor.invoke({"input": "What did I just ask you about?"})["output"]
# agent_executor.invoke({"input": "Will it look like a white Christmas there?"})["output"]


C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


llm initialized


> Entering new AgentExecutor chain...


C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Thought: Do I need to use a tool? No
AI: What's up! How can I assist you today?

> Finished chain.


In [3]:
agent_executor.invoke({"input": "What is the speed of light?"})["output"]




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: Speed of lightThe speed of light in vacuum, commonly denoted c, is a universal physical constant that is exactly equal to 299,792,458 metres per second (approximately 300,000 kilometres per second; 186,000 miles per second; 671 million miles per hour). [Note 3] According to the special theory of relativity, c is the upper limit for the speed at which ... The speed of light traveling through a vacuum is exactly 299,792,458 meters (983,571,056 feet) per second. That's about 186,282 miles per second — a universal constant known in equations as "c ... Speed of light, speed at which light waves propagate through different materials. In a vacuum, the speed of light is 299,792,458 meters per second. The speed of light is considered a fundamental constant of nature. Its significance is far broader than its role in describing a property of electromagnetic waves. The speed of light 

'The speed of light in a vacuum is approximately 299,792,458 meters per second, which is about 186,282 miles per second.'

In [7]:
class AgentExecutorInterface(gr.Interface):
    def __init__(self, agent_executor, title=None, description=None):
        inputs = gr.Textbox(lines=5, label="Enter your text here")
        outputs = gr.Textbox(label="Response will appear here")
        super().__init__(self.interact_with_agent, inputs=inputs, outputs=outputs, title=title, description=description)
        self.agent_executor = agent_executor

    def interact_with_agent(self, input_text):
        payload = {"input": input_text}
        return self.agent_executor.invoke(payload)["output"]
 
# Example usage
agent_executor_url = "YOUR_AGENT_EXECUTOR_ENDPOINT"  # Replace this with your agent executor endpoint
agent_interface = AgentExecutorInterface(agent_executor, title="Agent Executor", description="Interact with the agent executor using this interface")
agent_interface.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: What is the speed of light

Traceback (most recent call last):
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\gradio\routes.py", line 507, in predict
    output = await route_utils.call_process_api(
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\gradio\route_utils.py", line 219, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\gradio\blocks.py", line 1437, in process_api
    result = await self.call_function(
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\gradio\blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\anyio\_backends\_asyncio.py", line 2144, in run_sync_in



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: res

Traceback (most recent call last):
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\gradio\routes.py", line 507, in predict
    output = await route_utils.call_process_api(
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\gradio\route_utils.py", line 219, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\gradio\blocks.py", line 1437, in process_api
    result = await self.call_function(
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\gradio\blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "C:\Users\sujit\Desktop\ms_nlp\245\project\env\lib\site-packages\anyio\_backends\_asyncio.py", line 2144, in run_sync_in